# Projet MRO - COP

Dans ce NoteBook, on s'applique à trouver des solutions pour notre modélisation sur le probème d'attribution de fréquences.

## Bibliothèques

In [1]:
from pycsp3 import *
import json
import numpy as np


À utiliser à chaque fois que l'on veut lancer le code pour un nouveau fichier ou une nouvelle optimisation : cela permet de vider le contenu en amont des variables allouées pour la résolution du problème

In [2]:
clear()

## Initialisation de la modélisation

### Chargement des données

In [3]:
data='test.json'

### Fonction utile pour la contrainte de région

In [4]:
def fqc_par_region(E, R, stations, regions):
    res = []
    index_region = stations[0]['region']
    fqc_utilise = []
    for i in range(len(stations)):
        if stations[i]['region'] == index_region: 
            fqc_utilise.append(E[i])
            fqc_utilise.append(R[i])
        else:
            index_region+=1
            res.append(fqc_utilise)
            fqc_utilise = []
            fqc_utilise.append(E[i])
            fqc_utilise.append(R[i])
    res.append(fqc_utilise)     
    return res

### Chargemment des données et tableau utiles à l'écriture des contraintes

In [5]:
with open(data, 'r') as f:
    f = json.load(f)
    stations=f['stations']
    liaisons = f['liaisons']
    interferences = f['interferences']
    regions = f['regions']

### Création des variables de notre modélisation

In [6]:
E_fq = VarArray( size =len(stations), dom = lambda i : stations[i]["emetteur"] )
R_fq = VarArray( size =len(stations), dom = lambda i : stations[i]["recepteur"] )
nb_stations = len(E_fq)

In [7]:
list_fqc_par_region = fqc_par_region(E_fq,R_fq,stations,regions)

### Écriture des contraintes de notre modélisation

In [8]:
satisfy(
        (abs(R_fq[i] - E_fq[i]) == stations[i]["delta"] for i in range(nb_stations)),

        ((abs(E_fq[interferences[l]["x"]] - R_fq[interferences[l]["y"]])) >= interferences[l]["Delta"] for l in range(len(interferences))),

        ((abs(R_fq[interferences[l]["x"]] - E_fq[interferences[l]["y"]])) >= interferences[l]["Delta"] for l in range(len(interferences))),

        ((abs(R_fq[interferences[l]["x"]] - R_fq[interferences[l]["y"]])) >= interferences[l]["Delta"] for l in range(len(interferences))),

        ((abs(E_fq[interferences[l]["x"]] - E_fq[interferences[l]["y"]])) >= interferences[l]["Delta"] for l in range(len(interferences))),

        (E_fq[liaisons[i]['x']] == R_fq[liaisons[i]['y']] for i in range(len(liaisons))),

        (E_fq[liaisons[i]['y']] == R_fq[liaisons[i]['x']] for i in range(len(liaisons))),

        (NValues(list_fqc_par_region[k]) <= regions[k] for k in range(len(regions)))
    )

intension(function:eq(dist(R_fq[0],E_fq[0]),140))
intension(function:eq(dist(R_fq[1],E_fq[1]),140))
intension(function:eq(dist(R_fq[2],E_fq[2]),140))
intension(function:ge(dist(E_fq[0],R_fq[1]),20))
intension(function:ge(dist(E_fq[0],R_fq[2]),30))
intension(function:ge(dist(R_fq[0],E_fq[1]),20))
intension(function:ge(dist(R_fq[0],E_fq[2]),30))
intension(function:ge(dist(R_fq[0],R_fq[1]),20))
intension(function:ge(dist(R_fq[0],R_fq[2]),30))
intension(function:ge(dist(E_fq[0],E_fq[1]),20))
intension(function:ge(dist(E_fq[0],E_fq[2]),30))
intension(function:eq(E_fq[1],R_fq[2]))
intension(function:eq(E_fq[2],R_fq[1]))
nValues(list:[E_fq[0], R_fq[0]], condition:(le,2))
nValues(list:[E_fq[1], R_fq[1], E_fq[2], R_fq[2]], condition:(le,3))

### Différentes fonctions à optimiser (n'en charger qu'une seule)

#### Minimisation du nombre de fréquences utilisées

In [9]:
minimize(
    NValues(E_fq + R_fq)
)

#### Minimisation de la fréquence la plus élevée utilisée

In [10]:
minimize(
    Maximum(E_fq + R_fq)
)

#### Minimisation de la largeur de la bande de fréquences utilisées

In [11]:
minimize(
    Maximum(E_fq + R_fq) - Minimum(E_fq + R_fq)
)

## Résolution par les solveurs ACE et CHOCO

### Initialisation de nos solveurs

In [12]:
instance = compile()
ace = solver(ACE)
choco = solver(CHOCO)

### Calcul du temps mis en moyenne par ACE

In [13]:
%%timeit 
ace.solve(instance)

421 ms ± 6.52 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Calcul du temps mis en moyenne par CHOCO

In [14]:
%%timeit 
choco.solve(instance)

229 ms ± 8.32 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Résolution du problème par ACE

In [17]:
result = ace.solve(instance)
print("Result:", result)
if result is OPTIMUM:
    print(values(E_fq))
    print(values(R_fq))

Result: OPTIMUM
[14, 56, 196]
[154, 196, 56]


### Résolution du problème par CHOCO

In [16]:
result = choco.solve(instance)
print("Result:", result)
if result is OPTIMUM:
    print(values(E_fq))
    print(values(R_fq))

Result: OPTIMUM
[14, 196, 56]
[154, 56, 196]
